# Demo Notebook

Demo notebook showing how to find internal oceans and diffusive cells

In [ ]:
import xarray as xr
import numpy

import hvplot.xarray

import holoviews as hv

from scipy.ndimage import measurements
from scipy.signal import convolve2d

In [ ]:
ds = xr.open_dataset("/home/wesley/Downloads/ScotMuell30Ma.nc")
ds

In [ ]:
values = ds.ScotMuell31Ma
ocean = values < 0
ocean.hvplot('lon', 'lat')

In [ ]:
labeled_array, num_features = measurements.label(ocean)

# Replace continents with numpy.NaN
labeled_array = labeled_array.astype(object)
labeled_array[labeled_array==0] = numpy.NaN

# Draw
hv.Image(labeled_array).opts(tools=['hover'], width = 600).opts(clipping_colors = {'min': 'grey', 'max': 'black'})

In [ ]:
# Define template we are looking for passages
# Where only diffusion occurs this means we are looking
# for ocean passages one in width/height
# 1 => Ocean
# -1 => Land
# 0 = Indifferent
template = numpy.array([[0, 1, 0], 
                        [-1,1,-1], 
                        [0, 1, 0]])

# Theoretical max value when the template is found
# Note that 0s are considered wildcards so they are not taken into
# Account 
#TODO this only works on data arrays where the absolute values are 1
perfect_match = numpy.sum(numpy.abs(template))

# we recode the values of land to -1 as
# we did in the template
values = ocean.values.astype(int)
values[values == 0] = -1

# Create an empty array where we are going to stock the values
#TODO This could potentially by a binary array??
potential_points = numpy.empty(ocean.values.shape)
potential_points[:] = numpy.nan

# Mark points where there is only diffusion in longitude direction
convolvedh = convolve2d(values, template, 'same')
potential_points[convolvedh == perfect_match] = 1

# Mark points where there is only diffusion in latitude direction
convolvedv = convolve2d(values, template.T, 'same')
potential_points[convolvedv == perfect_match] = 1

hv.Image(potential_points).opts(tools=['hover'], width=400, clipping_colors={'NaN':'gray'}) + hv.Image(values.reshape(*ocean.values.shape)).opts(width=400, tools=['hover'])

In [ ]:
import pandas as pd
import holoviews as hv

from bokeh.sampledata import stocks
from holoviews.operation.timeseries import rolling, rolling_outlier_std

hv.extension('bokeh')

In [ ]:
import param
import panel as pn

variables = ['open', 'high', 'low', 'close', 'volume', 'adj_close']

class StockExplorer(param.Parameterized):

    rolling_window = param.Integer(default=10, bounds=(1, 365))
    
    symbol = param.ObjectSelector(default='AAPL', objects=stock_symbols)
    
    variable = param.ObjectSelector(default='adj_close', objects=variables)

    @param.depends('symbol', 'variable')
    def load_symbol(self):
        df = pd.DataFrame(getattr(stocks, self.symbol))
        df['date'] = df.date.astype('datetime64[ns]')
        return hv.Curve(df, ('date', 'Date'), self.variable).opts(framewise=True)
    
    def show(self):
        

In [ ]:
from holoviews.operation.datashader import datashade, dynspread
from holoviews.selection import link_selections

class AdvancedStockExplorer(StockExplorer):    

    datashade = param.Boolean(default=False)

    @param.depends('datashade')
    def view(self):
        stocks = hv.DynamicMap(self.load_symbol)

        # Apply rolling mean
        smoothed = rolling(stocks, rolling_window=self.param.rolling_window)
        if self.datashade:
            smoothed = dynspread(datashade(smoothed, aggregator='any')).opts(framewise=True)

        # Find outliers
        outliers = rolling_outlier_std(stocks, rolling_window=self.param.rolling_window).opts(
            width=600, color='red', marker='triangle', framewise=True)
        return pn.Row(pn.panel(self.param), link_selections(smoothed + outliers))

In [ ]:
explorer = AdvancedStockExplorer()
explorer.view()